# Actualizacion

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

# Diccionario actualizado con país, estado y coordenadas
paises_estados_coordenadas = {
    '1': 
        {'ESTADOID': '1', 'Coordenadas': '21.706,-102.318'}
}

url_general = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
parametros = {
    'date1': '2025-02-12',
    'date2': '2025-02-12',
    #'llave': 'NCJL7WWCS7UGFB9KKT7ZR8MYD'
}

dataframes = []

for pais, info in paises_estados_coordenadas.items():
    if isinstance(info, list):
        for estado_info in info:
            estado = estado_info['ESTADOID']
            coordenadas = estado_info['Coordenadas']
            url = f"{url_general}{coordenadas}/{parametros['date1']}/{parametros['date2']}?key={parametros['llave']}&include=days&unitGroup=metric"
            response = requests.get(url)
            if response.status_code == 200:
                weather_data = response.json()
                inicio = datetime.strptime(parametros['date1'], '%Y-%m-%d').date()
                fin = datetime.strptime(parametros['date2'], '%Y-%m-%d').date()
                fecha_actual = inicio
                fechas = []
                while fecha_actual <= fin:
                    fechas.append(fecha_actual)
                    fecha_actual += timedelta(days=1)
                dias = []
                for indice, fecha in enumerate(fechas):
                    fecha_str = fecha.strftime('%Y-%m-%d')
                    dia_data = weather_data['days'][indice]
                    dia_data['PAISID'] = pais
                    dia_data['ESTADOID'] = estado
                    dia_data['Fecha'] = fecha_str
                    dias.append(dia_data)
                data = pd.DataFrame(dias)
                dataframes.append(data)
            else:
                print(f'Error en la petición para {pais}, {estado}:', response.status_code)
    else:
        estado = info['ESTADOID']
        coordenadas = info['Coordenadas']
        url = f"{url_general}{coordenadas}/{parametros['date1']}/{parametros['date2']}?key={parametros['llave']}&include=days&unitGroup=metric"
        response = requests.get(url)
        if response.status_code == 200:
            weather_data = response.json()
            inicio = datetime.strptime(parametros['date1'], '%Y-%m-%d').date()
            fin = datetime.strptime(parametros['date2'], '%Y-%m-%d').date()
            fecha_actual = inicio
            fechas = []
            while fecha_actual <= fin:
                fechas.append(fecha_actual)
                fecha_actual += timedelta(days=1)
            dias = []
            for indice, fecha in enumerate(fechas):
                fecha_str = fecha.strftime('%Y-%m-%d')
                dia_data = weather_data['days'][indice]
                dia_data['PAISID'] = pais
                dia_data['ESTADOID'] = estado
                dia_data['Fecha'] = fecha_str
                dias.append(dia_data)
            data = pd.DataFrame(dias)
            dataframes.append(data)
        else:
            print(f'Error en la petición para {pais}, {estado}:', response.status_code)

data_final = pd.concat(dataframes, ignore_index=True)
data_final['TmpID'] = data_final['Fecha'].str.replace('-', '')
data_final['Fecha'] = pd.to_datetime(data_final['Fecha'], errors='coerce')

data_final.drop(['datetime', 'datetimeEpoch','Fecha'], axis=1, inplace=True)
#data_final.to_excel(r"internacionalclimaact.xlsx", index=False)

data_final

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,PAISID,ESTADOID,TmpID
0,26.9,9.1,17.6,26.1,8.1,17.5,3.4,41.4,0.0,0.0,0.0,None,0.0,0.0,50.4,14.4,219.5,1009.6,37.3,11.3,257.6,22.3,9.0,07:21:46,1739366506,18:45:35,1739407535,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,1,1,20250212


In [2]:
from dotenv import load_dotenv
import os
#from snowflake.connector import connect
from sqlalchemy import create_engine

# Cargar variables de entorno desde .env
load_dotenv()

# Conexión a Snowflake
SF_USER = os.getenv("SF_USER")
SF_PASSWORD = os.getenv("SF_PASSWORD")
SF_ACCOUNT = os.getenv("SF_ACCOUNT")
SF_DATABASE = os.getenv("SF_DATABASE")
SF_SCHEMA = os.getenv("SF_SCHEMA")
SF_SCHEMA = os.getenv("SF_SCHEMA")
SF_SCHEMA_CT = os.getenv("SF_SCHEMA_CT")
# SF_TABLE = os.getenv("SF_TABLE")

# Configura el motor de conexión
engine = create_engine(
    f'snowflake://{SF_USER}:{SF_PASSWORD}@{SF_ACCOUNT}'
)
conn = engine.connect()

## Datos que di de alta

In [22]:
df = pd.read_csv('coordenadas_paises.csv', encoding='latin-1')


In [23]:
from unidecode import unidecode

df[['PAIS', 'ESTADO']] = df[['PAIS', 'ESTADO']].applymap(lambda x: unidecode(str(x)).upper())


C:\Users\elsilva\AppData\Local\Temp\ipykernel_24828\1108333815.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['PAIS', 'ESTADO']] = df[['PAIS', 'ESTADO']].applymap(lambda x: unidecode(str(x)).upper())


In [25]:
df.to_csv('Datos_coordenadas_estados.csv', index=False)
df

,PAISID,RELPAISID,PAIS,IDESTADO,ESTADO,LATIDUD,LONGITUD
0,1,1,MEXICO,1.0,AGUASCALIENTES,21.70600,-102.31800
1,1,1,MEXICO,2.0,BAJA CALIFORNIA,32.80700,-115.44600
2,1,1,MEXICO,3.0,BAJA CALIFORNIA SUR,24.16700,-110.30000
3,1,1,MEXICO,4.0,CAMPECHE,19.81700,-90.50000
4,1,1,MEXICO,7.0,CHIAPAS,16.75000,-92.63300
...,...,...,...,...,...,...,...
401,24,180,PARAGUAY,NaN,AMAMBAY,-26.18300,-56.36700
402,24,180,PARAGUAY,NaN,CANINDEYU,-24.03300,-54.35000
403,24,180,PARAGUAY,NaN,PRESIDENTE HAYES,-25.09306,-57.52361
404,24,180,PARAGUAY,NaN,BOQUERON,-22.02000,-60.60000


In [44]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Cargar el DataFrame (ajusta la ruta según el archivo)
df = df_cordenadas # O usa pd.read_excel("archivo.xlsx")

# Definir la URL base de la API
url_general = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

# Definir las fechas de consulta
parametros = {
    'date1': '2025-02-12',
    'date2': '2025-02-12',
    'llave': #'NCJL7WWCS7UGFB9KKT7ZR8MYD' # Reemplaza con tu clave real
}

# Lista para almacenar los DataFrames de cada consulta
dataframes = []

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    pais = row['pais']
    estado = row['idestado']
    coordenadas = f"{row['latitud']},{row['longitud']}"

    # Construir la URL para la solicitud API
    url = f"{url_general}{coordenadas}/{parametros['date1']}/{parametros['date2']}?key={parametros['llave']}&include=days&unitGroup=metric"

    # Realizar la solicitud a la API
    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()
        
        # Generar la lista de fechas dentro del rango
        inicio = datetime.strptime(parametros['date1'], '%Y-%m-%d').date()
        fin = datetime.strptime(parametros['date2'], '%Y-%m-%d').date()
        fechas = [inicio + timedelta(days=i) for i in range((fin - inicio).days + 1)]
        
        # Extraer los datos climáticos
        dias = []
        for indice, fecha in enumerate(fechas):
            fecha_str = fecha.strftime('%Y-%m-%d')
            dia_data = weather_data['days'][indice]
            dia_data['pais'] = pais
            dia_data['idestado'] = estado
            dia_data['Fecha'] = fecha_str
            dias.append(dia_data)
        
        # Crear un DataFrame con los datos obtenidos
        data = pd.DataFrame(dias)
        dataframes.append(data)

    else:
        print(f'Error en la petición para {pais}, {estado}:', response.status_code)

# Concatenar todos los DataFrames en uno solo
data_final = pd.concat(dataframes, ignore_index=True)

# Crear un identificador temporal basado en la fecha
data_final['TmpID'] = data_final['Fecha'].str.replace('-', '')

# Convertir la columna de fecha a tipo datetime
data_final['Fecha'] = pd.to_datetime(data_final['Fecha'], errors='coerce')

# Eliminar columnas innecesarias
data_final.drop(['datetime', 'datetimeEpoch', 'Fecha'], axis=1, inplace=True)

# Guardar los datos en un archivo si es necesario
# data_final.to_excel("clima_actualizado.xlsx", index=False)

# Mostrar el DataFrame final
data_final


C:\Users\elsilva\AppData\Local\Temp\ipykernel_24828\3631310387.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_final = pd.concat(dataframes, ignore_index=True)


,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,pais,idestado,TmpID
0,26.9,9.1,17.6,26.1,8.1,17.5,3.4,41.4,0.0,0.0,0.00,None,0.0,0.0,50.4,14.4,219.5,1009.6,37.3,11.3,257.6,22.3,9.0,07:21:46,1739366506,18:45:35,1739407535,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,MEXICO,1.0,20250212
1,20.7,13.3,16.4,20.7,13.3,16.4,4.7,46.2,0.0,0.0,0.00,None,0.0,0.0,62.6,39.2,265.4,1012.1,56.6,16.0,137.3,11.9,7.0,06:27:29,1739370449,17:25:02,1739409902,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KNYL, 74003503145, KNJK, BTTC1, KIPL, CI087, ...",obs,MEXICO,2.0,20250212
2,24.3,12.4,18.2,24.3,12.4,18.2,10.9,65.2,0.0,0.0,0.00,None,0.0,0.0,34.2,22.0,229.3,1014.1,6.9,21.0,244.9,21.1,9.0,06:56:24,1739368584,18:14:50,1739409290,0.5,Clear,Clear conditions throughout the day.,clear-day,"[MMLP, 76405099999, 76405599999, remote]",obs,MEXICO,3.0,20250212
3,35.9,22.0,28.4,45.3,22.0,31.2,22.3,71.3,0.0,0.0,0.00,None,0.0,0.0,47.5,25.6,129.2,1009.3,14.2,10.1,222.5,19.2,8.0,06:32:28,1739363548,18:00:18,1739404818,0.5,Clear,Clear conditions throughout the day.,clear-day,"[MMCP, remote, 76695099999]",obs,MEXICO,4.0,20250212
4,22.2,12.4,16.8,22.2,12.4,16.8,10.6,68.6,0.0,0.0,0.00,None,0.0,0.0,44.3,7.2,197.8,1009.8,51.9,6.7,246.6,21.2,9.0,06:37:48,1739363868,18:12:01,1739405521,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[76845099999, MMTG, 76843099999, remote, 76848...",obs,MEXICO,7.0,20250212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,37.1,24.3,29.7,42.7,24.3,32.2,21.8,64.2,1.9,100.0,8.33,[rain],0.0,0.0,40.7,18.4,18.0,1008.6,49.3,20.0,289.7,24.8,10.0,06:28:40,1739352520,19:30:25,1739399425,0.5,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"[86251099999, 86234099999, 86268099999, remote]",obs,PARAGUAY,NaN,20250212
402,30.6,21.7,25.1,32.5,21.7,25.8,20.8,78.5,3.6,100.0,29.17,[rain],0.0,0.0,34.2,14.8,23.0,1010.6,29.5,20.0,303.5,26.2,10.0,06:23:13,1739352193,19:19:47,1739398787,0.5,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[86210099999, 86896099999, 86895099999]",obs,PARAGUAY,NaN,20250212
403,39.0,25.0,31.8,41.8,25.0,34.0,20.2,52.5,0.0,0.0,0.00,None,0.0,0.0,44.6,25.9,17.2,1007.5,35.5,11.6,308.8,26.6,10.0,06:34:38,1739352878,19:33:44,1739399624,0.5,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[SGAS, 86221099999, 86218099999]",obs,PARAGUAY,NaN,20250212
404,39.9,24.9,32.0,43.8,24.9,34.6,21.0,54.4,0.0,0.0,0.00,None,0.0,0.0,48.2,19.4,19.3,1006.4,27.6,15.0,297.2,25.7,10.0,06:50:35,1739353835,19:42:26,1739400146,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[86068099999, SGME, remote]",obs,PARAGUAY,NaN,20250212


In [47]:
data_final.to_csv('datos_clima.csv', index=False)

In [46]:
data_final

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,pais,idestado,TmpID
0,26.9,9.1,17.6,26.1,8.1,17.5,3.4,41.4,0.0,0.0,0.00,None,0.0,0.0,50.4,14.4,219.5,1009.6,37.3,11.3,257.6,22.3,9.0,07:21:46,1739366506,18:45:35,1739407535,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,MEXICO,1.0,20250212
1,20.7,13.3,16.4,20.7,13.3,16.4,4.7,46.2,0.0,0.0,0.00,None,0.0,0.0,62.6,39.2,265.4,1012.1,56.6,16.0,137.3,11.9,7.0,06:27:29,1739370449,17:25:02,1739409902,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KNYL, 74003503145, KNJK, BTTC1, KIPL, CI087, ...",obs,MEXICO,2.0,20250212
2,24.3,12.4,18.2,24.3,12.4,18.2,10.9,65.2,0.0,0.0,0.00,None,0.0,0.0,34.2,22.0,229.3,1014.1,6.9,21.0,244.9,21.1,9.0,06:56:24,1739368584,18:14:50,1739409290,0.5,Clear,Clear conditions throughout the day.,clear-day,"[MMLP, 76405099999, 76405599999, remote]",obs,MEXICO,3.0,20250212
3,35.9,22.0,28.4,45.3,22.0,31.2,22.3,71.3,0.0,0.0,0.00,None,0.0,0.0,47.5,25.6,129.2,1009.3,14.2,10.1,222.5,19.2,8.0,06:32:28,1739363548,18:00:18,1739404818,0.5,Clear,Clear conditions throughout the day.,clear-day,"[MMCP, remote, 76695099999]",obs,MEXICO,4.0,20250212
4,22.2,12.4,16.8,22.2,12.4,16.8,10.6,68.6,0.0,0.0,0.00,None,0.0,0.0,44.3,7.2,197.8,1009.8,51.9,6.7,246.6,21.2,9.0,06:37:48,1739363868,18:12:01,1739405521,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[76845099999, MMTG, 76843099999, remote, 76848...",obs,MEXICO,7.0,20250212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,37.1,24.3,29.7,42.7,24.3,32.2,21.8,64.2,1.9,100.0,8.33,[rain],0.0,0.0,40.7,18.4,18.0,1008.6,49.3,20.0,289.7,24.8,10.0,06:28:40,1739352520,19:30:25,1739399425,0.5,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"[86251099999, 86234099999, 86268099999, remote]",obs,PARAGUAY,NaN,20250212
402,30.6,21.7,25.1,32.5,21.7,25.8,20.8,78.5,3.6,100.0,29.17,[rain],0.0,0.0,34.2,14.8,23.0,1010.6,29.5,20.0,303.5,26.2,10.0,06:23:13,1739352193,19:19:47,1739398787,0.5,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"[86210099999, 86896099999, 86895099999]",obs,PARAGUAY,NaN,20250212
403,39.0,25.0,31.8,41.8,25.0,34.0,20.2,52.5,0.0,0.0,0.00,None,0.0,0.0,44.6,25.9,17.2,1007.5,35.5,11.6,308.8,26.6,10.0,06:34:38,1739352878,19:33:44,1739399624,0.5,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[SGAS, 86221099999, 86218099999]",obs,PARAGUAY,NaN,20250212
404,39.9,24.9,32.0,43.8,24.9,34.6,21.0,54.4,0.0,0.0,0.00,None,0.0,0.0,48.2,19.4,19.3,1006.4,27.6,15.0,297.2,25.7,10.0,06:50:35,1739353835,19:42:26,1739400146,0.5,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[86068099999, SGME, remote]",obs,PARAGUAY,NaN,20250212


In [ ]:
df_cordenadas

In [7]:
from sqlalchemy import create_engine
# Configura el motor de conexión
engine = create_engine(
    f'snowflake://{SF_USER}:{SF_PASSWORD}@{SF_ACCOUNT}'
)
conn = engine.connect()

coordenadas_regiones =''' 
SELECT * FROM DEV_STG.GNM_CT.CAT_REGIONES_COORDENADAS'''

df_cordenadas = pd.read_sql(coordenadas_regiones, conn)
df_cordenadas

,paisid,relpaisid,pais,idestado,estado,latitud,longitud
0,1,1,MEXICO,1.0,AGUASCALIENTES,21.70600,-102.31800
1,1,1,MEXICO,2.0,BAJA CALIFORNIA,32.80700,-115.44600
2,1,1,MEXICO,3.0,BAJA CALIFORNIA SUR,24.16700,-110.30000
3,1,1,MEXICO,4.0,CAMPECHE,19.81700,-90.50000
4,1,1,MEXICO,7.0,CHIAPAS,16.75000,-92.63300
...,...,...,...,...,...,...,...
401,24,180,PARAGUAY,NaN,AMAMBAY,-26.18300,-56.36700
402,24,180,PARAGUAY,NaN,CANINDEYU,-24.03300,-54.35000
403,24,180,PARAGUAY,NaN,PRESIDENTE HAYES,-25.09306,-57.52361
404,24,180,PARAGUAY,NaN,BOQUERON,-22.02000,-60.60000


In [5]:
df_cordenadas

NameError: name 'df_cordenadas' is not defined

In [17]:
import pandas as pd
import requests
from datetime import datetime, timedelta

coordenadas_regiones =''' 
SELECT * FROM DEV_STG.GNM_CT.CAT_REGIONES_COORDENADAS'''

df_cordenadas = pd.read_sql(coordenadas_regiones, conn)


# URL base de la API
URL_BASE = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
# Fechas de consulta
DATE_START = '2025-02-15'  # Fecha de inicio
DATE_END = '2025-02-15'    # Fecha de fin (puede ser igual a DATE_START para un solo día)
API_KEY = 'NCJL7WWCS7UGFB9KKT7ZR8MYD'  # Reemplaza con tu clave real
# Función para generar una lista de fechas dentro del rango especificado
def generar_fechas(inicio, fin):
    try:
        inicio = datetime.strptime(inicio, '%Y-%m-%d')
        fin = datetime.strptime(fin, '%Y-%m-%d')
        # genera los días que están entre inicio i fin
        return [(inicio + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((fin - inicio).days + 1)]
    except Exception as e:
        print(f"Error al generar fechas: {e}")
        return []
# Función para obtener datos climáticos y validar las fechas esperadas
def obtener_datos_clima(row):
    try:
        coordenadas = f"{row['latitud']},{row['longitud']}"
        url = f"{URL_BASE}{coordenadas}/{DATE_START}/{DATE_END}?key={API_KEY}&include=days&unitGroup=metric"
        # Realizar la solicitud HTTP
        try:
            response = requests.get(url)
            response.raise_for_status()  # Lanza una excepción si el código de estado no es 200
        except requests.exceptions.RequestException as e:
            print(f"Error en la petición HTTP para {row['pais']}, {row['idestado']}: {e}")
            return None
        # Procesar la respuesta JSON
        try:
            weather_data = response.json()
        except ValueError as e:
            print(f"Error al decodificar JSON para {row['pais']}, {row['idestado']}: {e}")
            return None
        # dias es una lista de diccionarios obtenidos de la api de clima
        # obtener la llave days que contiene los dtos que nos interesan
        dias = weather_data.get('days', [])
        # Generar las fechas esperadas en el rango solicitado
        fechas_esperadas = generar_fechas(DATE_START, DATE_END)
        
        dias_procesados = []
        for fecha_esperada in fechas_esperadas:
            # va obtenido un diccionario para el día correspondiente
            dia_encontrado = next((dia for dia in dias if dia['datetime'] == fecha_esperada), None)
            # añadir los datos adicionales que no vienen desde la api
            if dia_encontrado:
                dia_encontrado['pais'] = row['pais']
                dia_encontrado['idestado'] = row['idestado']
                dia_encontrado['Fecha'] = datetime.strptime(fecha_esperada, '%Y-%m-%d')
                dia_encontrado['TmpID'] = fecha_esperada.replace('-', '')
                print(dia_encontrado)
                dias_procesados.append(dia_encontrado)
            else:
                print(f"Fecha {fecha_esperada} no encontrada para {row['pais']}, {row['idestado']}")
        return dias_procesados
    except Exception as e:
        print(f"Error inesperado al procesar {row['pais']}, {row['idestado']}: {e}")
        return None
# Aplicar la función a cada fila del DataFrame y expandir los resultados en una nueva lista
try:
    datos_clima = df_cordenadas.apply(obtener_datos_clima, axis=1).dropna()
except Exception as e:
    print(f"Error al aplicar la función a las filas del DataFrame: {e}")
    exit()
# Convertir la lista de listas en un solo DataFrame
try:
    data_final = pd.DataFrame([item for sublist in datos_clima for item in sublist])
except Exception as e:
    print(f"Error al convertir los datos en DataFrame: {e}")
    exit()




{'datetime': '2025-02-15', 'datetimeEpoch': 1739599200, 'tempmax': 29.2, 'tempmin': 8.1, 'temp': 18.1, 'feelslikemax': 27.5, 'feelslikemin': 8.1, 'feelslike': 17.9, 'dew': 1.4, 'humidity': 38.5, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': None, 'snow': 0.0, 'snowdepth': 0.0, 'windgust': 46.4, 'windspeed': 23.8, 'winddir': 224.6, 'pressure': 1014.1, 'cloudcover': 58.0, 'visibility': 11.8, 'solarradiation': 254.9, 'solarenergy': 22.1, 'uvindex': 9.0, 'sunrise': '07:19:56', 'sunriseEpoch': 1739625596, 'sunset': '18:47:10', 'sunsetEpoch': 1739666830, 'moonphase': 0.6, 'conditions': 'Partially cloudy', 'description': 'Partly cloudy throughout the day.', 'icon': 'partly-cloudy-day', 'stations': ['MMAS', '76571099999', '76573099999', 'remote'], 'source': 'obs', 'pais': 'MEXICO', 'idestado': 1.0, 'Fecha': datetime.datetime(2025, 2, 15, 0, 0), 'TmpID': '20250215'}
{'datetime': '2025-02-15', 'datetimeEpoch': 1739606400, 'tempmax': 23.0, 'tempmin': 7.4, 'temp': 14.9, 'feel

# El último

In [4]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Cargar el DataFrame desde la base de datos
coordenadas_regiones = '''SELECT * FROM DEV_STG.GNM_CT.CAT_REGIONES_COORDENADAS'''
df_cordenadas = pd.read_sql(coordenadas_regiones, conn)
df_cordenadas = df_cordenadas.head(1)
# URL base de la API
URL_BASE = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
DATE_START = '2025-03-03'  # Fecha de inicio
DATE_END = '2025-03-03'    # Fecha de fin
API_KEY = 'NCJL7WWCS7UGFB9KKT7ZR8MYD'  # Reemplaza con tu clave real

# Función para generar una lista de fechas dentro del rango especificado
def generar_fechas(inicio, fin):
    try:
        inicio = datetime.strptime(inicio, '%Y-%m-%d')
        fin = datetime.strptime(fin, '%Y-%m-%d')
        return [(inicio + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((fin - inicio).days + 1)]
    except Exception as e:
        print(f"Error al generar fechas: {e}")
        return []

# Función para obtener datos climáticos
def obtener_datos_clima(row):
    try:
        coordenadas = f"{row['latitud']},{row['longitud']}"
        url = f"{URL_BASE}{coordenadas}/{DATE_START}/{DATE_END}?key={API_KEY}&include=days&unitGroup=metric"
        response = requests.get(url)
        # Lanza una excepción si el código de estado no es 200
        response.raise_for_status()
        weather_data = response.json()
        print(weather_data)
        # dias es una lista de diccionarios obtenidos de la api de clima
        # obtener la llave days que contiene los dtos que nos interesan
        dias = weather_data.get('days', [])
        fechas_esperadas = generar_fechas(DATE_START, DATE_END)
        # almacena los días encontrados
        dias_procesados = []
        for fecha_esperada in fechas_esperadas:
            # va obtenido un diccionario para el día correspondiente
            dia_encontrado = next((dia for dia in dias if dia['datetime'] == fecha_esperada), None)
             # añadir los datos adicionales que no vienen desde la api
            if dia_encontrado:
                dia_encontrado['paisid'] = row['paisid']
                dia_encontrado['idestado'] = row['idestado']
                #dia_encontrado['Fecha'] = datetime.strptime(fecha_esperada, '%Y-%m-%d')
                dia_encontrado['TmpID'] = fecha_esperada.replace('-', '')
                dias_procesados.append(dia_encontrado)
            else:
                print(f"Fecha {fecha_esperada} no encontrada para {row['pais']}, {row['idestado']}")
        return dias_procesados
    except Exception as e:
        print(f"Error inesperado al procesar {row['pais']}, {row['idestado']}: {e}")
        return None

# Paralelización de solicitudes HTTP
def procesar_filas_paralelamente(df):
    resultados = []
    with ThreadPoolExecutor(max_workers=10) as executor:  # Ajusta el número de workers según tu sistema
        futures = {executor.submit(obtener_datos_clima, row): row for _, row in df.iterrows()}
        for future in as_completed(futures):
            try:
                resultado = future.result()
                if resultado:
                    resultados.extend(resultado)
            except Exception as e:
                print(f"Error en una tarea: {e}")
    return resultados

# Procesar el DataFrame en paralelo
try:
    datos_clima = procesar_filas_paralelamente(df_cordenadas)
except Exception as e:
    print(f"Error al procesar el DataFrame: {e}")
    exit()

data_final = pd.DataFrame(datos_clima)
data_final.drop(columns=['datetime', 'datetimeEpoch'], inplace=True)



{'queryCost': 1, 'latitude': 21.706, 'longitude': -102.318, 'resolvedAddress': '21.706,-102.318', 'address': '21.706,-102.318', 'timezone': 'America/Mexico_City', 'tzoffset': -6.0, 'days': [{'datetime': '2025-03-03', 'datetimeEpoch': 1740981600, 'tempmax': 29.5, 'tempmin': 12.4, 'temp': 20.4, 'feelslikemax': 27.7, 'feelslikemin': 12.4, 'feelslike': 20.0, 'dew': -6.1, 'humidity': 17.3, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': None, 'snow': 0.0, 'snowdepth': 0.0, 'windgust': 34.2, 'windspeed': 30.2, 'winddir': 258.0, 'pressure': 1013.5, 'cloudcover': 50.6, 'visibility': 24.1, 'solarradiation': 287.7, 'solarenergy': 25.1, 'uvindex': 10.0, 'severerisk': 10.0, 'sunrise': '07:08:04', 'sunriseEpoch': 1741007284, 'sunset': '18:54:29', 'sunsetEpoch': 1741049669, 'moonphase': 0.13, 'conditions': 'Partially cloudy', 'description': 'Becoming cloudy in the afternoon.', 'icon': 'partly-cloudy-day', 'stations': ['remote'], 'source': 'comb'}]}


In [8]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TEMP_MAX                 1 non-null      float64
 1   TEMP_MIN                 1 non-null      float64
 2   TEMP_PROMEDIO            1 non-null      float64
 3   SENS_TERM_MAX            1 non-null      float64
 4   SENS_TERM_MIN            1 non-null      float64
 5   SENS_TERM_PROMEDIO       1 non-null      float64
 6   PUNTO_ROCIO              1 non-null      float64
 7   HUMEDAD                  1 non-null      float64
 8   PRECIPITACION            1 non-null      float64
 9   PROB_PRECIPITACION       1 non-null      float64
 10  COBERTURA_PRECIPITACION  1 non-null      float64
 11  TIPO_PRECIPITACION       0 non-null      object 
 12  NIEVE                    1 non-null      float64
 13  PROFUNDIDAD_NIEVE        1 non-null      float64
 14  RAFAGA_VIENTO            1 non

In [17]:



query_existentes = f"SELECT TMP_ID, PAIS_ID, ESTADO_ID FROM DEV_STG.GNM.HIST_CLIMA"
df_existente = pd.read_sql(query_existentes, conn)
df_existente['tmp_id']= df_existente['tmp_id'].astype(str)
df_existente.columns = df_existente.columns.str.upper()
# Convertir registros existentes a un conjunto de tuplas
merge = data_final.merge(df_existente, on=['TMP_ID','PAIS_ID','ESTADO_ID'],
                    how='left', indicator=True)

In [19]:
# Filtrar los registros nuevos de forma más rápida
df_a_insertar =  merge[merge['_merge']!='both']
df_a_insertar


,TEMP_MAX,TEMP_MIN,TEMP_PROMEDIO,SENS_TERM_MAX,SENS_TERM_MIN,SENS_TERM_PROMEDIO,PUNTO_ROCIO,HUMEDAD,PRECIPITACION,PROB_PRECIPITACION,...,FASE_LUNAR,CONDICIONES,DESCRIPCION,ICONO,ESTACIONES,FUENTE,PAIS_ID,ESTADO_ID,TMP_ID,_merge


In [7]:
# Diccionario de mapeo de nombres de columnas
column_mapping = {
    "TmpID": "TMP_ID",
    "paisid": "PAIS_ID",
    "idestado": "ESTADO_ID",
    "tempmax": "TEMP_MAX",
    "tempmin": "TEMP_MIN",
    "temp": "TEMP_PROMEDIO",
    "feelslikemax": "SENS_TERM_MAX",
    "feelslikemin": "SENS_TERM_MIN",
    "feelslike": "SENS_TERM_PROMEDIO",
    "dew": "PUNTO_ROCIO",
    "humidity": "HUMEDAD",
    "precip": "PRECIPITACION",
    "precipprob": "PROB_PRECIPITACION",
    "precipcover": "COBERTURA_PRECIPITACION",
    "preciptype": "TIPO_PRECIPITACION",
    "snow": "NIEVE",
    "snowdepth": "PROFUNDIDAD_NIEVE",
    "windgust": "RAFAGA_VIENTO",
    "windspeed": "VELOCIDAD_VIENTO",
    "winddir": "DIRECCION_VIENTO",
    "pressure": "PRESION",
    "cloudcover": "COBERTURA_NUBOSA",
    "visibility": "VISIBILIDAD",
    "solarradiation": "RADIACION_SOLAR",
    "solarenergy": "ENERGIA_SOLAR",
    "uvindex": "INDICE_UV",
    "sunrise": "AMANECER",
    "sunriseEpoch": "AMANECER_EPOCH",
    "sunset": "OCASO",
    "sunsetEpoch": "OCASO_EPOCH",
    "moonphase": "FASE_LUNAR",
    "conditions": "CONDICIONES",
    "description": "DESCRIPCION",
    "icon": "ICONO",
    "stations": "ESTACIONES",
    "source": "FUENTE"
}

#Renombrar las columnas del DataFrame
data_final.rename(columns=column_mapping, inplace=True)

In [38]:
data_final

,TEMP_MAX,TEMP_MIN,TEMP_PROMEDIO,SENS_TERM_MAX,SENS_TERM_MIN,SENS_TERM_PROMEDIO,PUNTO_ROCIO,HUMEDAD,PRECIPITACION,PROB_PRECIPITACION,COBERTURA_PRECIPITACION,TIPO_PRECIPITACION,NIEVE,PROFUNDIDAD_NIEVE,RAFAGA_VIENTO,VELOCIDAD_VIENTO,DIRECCION_VIENTO,PRESION,COBERTURA_NUBOSA,VISIBILIDAD,RADIACION_SOLAR,ENERGIA_SOLAR,INDICE_UV,severerisk,AMANECER,AMANECER_EPOCH,OCASO,OCASO_EPOCH,FASE_LUNAR,CONDICIONES,DESCRIPCION,ICONO,ESTACIONES,FUENTE,PAIS_ID,ESTADO_ID,TMP_ID
0,29.5,12.4,20.4,27.7,12.4,20.0,-5.7,17.6,0.0,0.0,0.0,None,0.0,0.0,34.2,30.2,255.5,1013.5,46.7,24.1,291.1,25.4,10.0,10.0,07:08:04,1741007284,18:54:29,1741049669,0.13,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,[remote],comb,1,1.0,20250303


In [18]:
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
from sqlalchemy.exc import SQLAlchemyError
from snowflake.connector.errors import DatabaseError, ProgrammingError
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Conectarse a Snowflake con el conector oficial
def get_snowflake_connection():
        return snowflake.connector.connect(
            user=SF_USER,
            password=SF_PASSWORD,
            account=SF_ACCOUNT,
            database=SF_DATABASE,
            schema=SF_SCHEMA
        )


# Conectarse a Snowflake con el conector oficial
def insertar_sf(df):
    conn = get_snowflake_connection()
    if not conn:
        print("No se pudo establecer la conexión. Abortando operación.")
        return
    try:
        success, num_chunks, num_rows, output = write_pandas(conn, df,"HIST_CLIMA")  
        if success:
            print(f"Insertados {num_rows} filas en la tabla HIST_CLIMA")
        else:
            print(f"La operación no fue completamente exitosa. Salida: {output}")
    except snowflake.connector.errors.ProgrammingError as e:
        print(f"Error en la ejecución SQL: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    finally:
        conn.close()



In [1]:
data_final

NameError: name 'data_final' is not defined

In [25]:
insertar_sf(data_final)

Insertados 1 filas en la tabla HIST_CLIMA


In [ ]:


data_final


,TEMP_MAX,TEMP_MIN,TEMP_PROMEDIO,SENS_TERM_MAX,SENS_TERM_MIN,SENS_TERM_PROMEDIO,PUNTO_ROCIO,HUMEDAD,PRECIPITACION,PROB_PRECIPITACION,COBERTURA_PRECIPITACION,TIPO_PRECIPITACION,NIEVE,PROFUNDIDAD_NIEVE,RAFAGA_VIENTO,VELOCIDAD_VIENTO,DIRECCION_VIENTO,PRESION,COBERTURA_NUBOSA,VISIBILIDAD,RADIACION_SOLAR,ENERGIA_SOLAR,INDICE_UV,AMANECER,AMANECER_EPOCH,OCASO,OCASO_EPOCH,FASE_LUNAR,CONDICIONES,DESCRIPCION,ICONO,ESTACIONES,FUENTE,PAIS_ID,ESTADO_ID,TMP_ID
0,29.2,8.1,18.1,27.5,8.1,17.9,1.4,38.5,0.0,0.0,0.0,None,0.0,0.0,46.4,23.8,224.6,1014.1,58.0,11.8,254.9,22.1,9.0,07:19:56,1739625596,18:47:10,1739666830,0.6,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,1,1.0,20250215


In [7]:
len(data_final.columns)

37

In [54]:
data_final

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,pais,idestado,Fecha,TmpID
0,2025-02-12,1739340000,26.9,9.1,17.6,26.1,8.1,17.5,3.4,41.4,0.0,0.0,0.0,None,0.0,0.0,50.4,14.4,219.5,1009.6,37.3,11.3,257.6,22.3,9.0,07:21:46,1739366506,18:45:35,1739407535,0.50,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,MEXICO,1.0,2025-02-12,20250212
1,2025-02-13,1739426400,28.1,7.1,17.2,26.8,7.1,17.0,3.6,46.1,0.0,0.0,0.0,None,0.0,0.0,32.8,7.2,174.8,1014.5,41.6,11.1,256.5,22.1,9.0,07:21:10,1739452870,18:46:07,1739493967,0.54,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999]",obs,MEXICO,1.0,2025-02-13,20250213
2,2025-02-14,1739512800,26.9,10.2,18.4,26.7,10.2,18.3,5.5,48.9,0.0,0.0,0.0,None,0.0,0.0,32.8,10.8,210.4,1016.7,44.3,10.9,231.5,20.0,8.0,07:20:34,1739539234,18:46:39,1739580399,0.57,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,MEXICO,1.0,2025-02-14,20250214
3,2025-02-15,1739599200,29.2,8.1,18.1,27.5,8.1,17.9,1.4,38.5,0.0,0.0,0.0,None,0.0,0.0,46.4,23.8,224.6,1014.1,58.0,11.8,254.9,22.1,9.0,07:19:56,1739625596,18:47:10,1739666830,0.60,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[MMAS, 76571099999, 76573099999, remote]",obs,MEXICO,1.0,2025-02-15,20250215


In [6]:
print(os.getenv("SF_SCHEMA"))

GNM


In [5]:
data_final.to_excel('actual_columnas.xlsx', index=False)